In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/intro-to-pytorch/intro-to-pytorch.zip
!unzip -qq intro-to-pytorch.zip
%cd intro-to-pytorch

--2023-10-11 04:19:05--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/intro-to-pytorch/intro-to-pytorch.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.218.234.73, 52.218.220.97, 52.92.228.58, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.218.234.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2372 (2.3K) [application/zip]
Saving to: ‘intro-to-pytorch.zip’

intro-to-pytorch.zi 100%[===================>]   2.32K  --.-KB/s    in 0s      

2023-10-11 04:19:06 (105 MB/s) - ‘intro-to-pytorch.zip’ saved [2372/2372]

/content/intro-to-pytorch


In [ ]:
from collections import OrderedDict
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import torch.nn as nn
import torch

In [ ]:
def get_training_model(inFeatures=4, hiddenDim=8, nbClasses = 3):
  mlpModel = nn.Sequential(OrderedDict([
      ("hidden_layer_1", nn.Linear(inFeatures, hiddenDim)),
      ("activation_1", nn.ReLU()),
      ("Output_layer", nn.Linear(hiddenDim, nbClasses))
  ]))
  return mlpModel

In [ ]:
def next_batch(inputs, targets, batchSize):
  for i in range(0, inputs.shape[0], batchSize):
    yield (inputs[i:i + batchSize], targets[i:i+batchSize])

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-2
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'
print("[INFO] training using {}...".format(DEVICE))

[INFO] training using cpu...


In [ ]:
print("[INFO] preparing data...")
(X, y) = make_blobs(n_samples=1000, n_features=4, centers=3, cluster_std=2.5, random_state=95)
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.15, random_state=95)

trainX = torch.from_numpy(trainX).float()
testX = torch.from_numpy(testX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()

[INFO] preparing data...


In [ ]:
mlp = get_training_model().to(DEVICE)
print(mlp)

opt = SGD(mlp.parameters(), lr=LR)
lossFunc = nn.CrossEntropyLoss()

Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=8, bias=True)
  (activation_1): ReLU()
  (Output_layer): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:
for epochs in range(0, EPOCHS):
  print("[INFO] epoch: {}...".format(epoch + 1))

  trainLoss = 0
  trainACC = 0
  samples = 0
  mlp.train()

  for (batchX, batchY) in next_batch(trainX, trainY, BATCH_SIZE):
    (batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))

    predictions = mlp(batchX)
    loss = lossFunc(predictions, batchY.long())

    opt.zero_grad()
    loss.backward()
    opt.step()

    trainLoss += loss.item() * batchY.size(0)
    trainACC += (predictions.max(1)[1] == batchY).sum().item()
    samples += batchY.size(0)

    trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"

    print(trainTemplate.format(epoch + 1, (trainLoss / samples),
                               (trainACC / samples)))

    testLoss = 0
    testACC = 0
    samples = 0
    mlp.eval()

    with torch.no_grad():
